In [2]:
import torch
from torch import Tensor
import torch.nn as nn
import math

from botorch.models import SingleTaskGP
from botorch.test_functions import Hartmann
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.generation import get_best_candidates, gen_candidates_torch
from botorch.optim import gen_batch_initial_conditions
from botorch.acquisition import qExpectedImprovement, qUpperConfidenceBound
from botorch.sampling import SobolQMCNormalSampler
import os
from typing import Any, Callable, Dict, List, NoReturn, Optional, Tuple, Type, Union
from botorch.test_functions.base import BaseTestProblem
from botorch.acquisition.acquisition import AcquisitionFunction
from botorch.acquisition.cached_cholesky import CachedCholeskyMCSamplerMixin # Updated import
from botorch.acquisition.objective import (
    IdentityMCObjective,
    MCAcquisitionObjective,
    PosteriorTransform,
)
from botorch.acquisition.utils import prune_inferior_points
from botorch.exceptions.errors import UnsupportedError
from botorch.models.model import Model
from botorch.posteriors.posterior import Posterior
from botorch.sampling.base import MCSampler
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils.transforms import (
    concatenate_pending_points,
    match_batch_shape,
    t_batch_mode_transform,
)
from botorch.acquisition.monte_carlo import MCAcquisitionFunction




def clamp_tensor(tensor: torch.Tensor, bounds: torch.Tensor) -> torch.Tensor:
    """
    Clamps the values of a tensor within given bounds.

    Args:
        tensor (Tensor): The tensor to clamp.
        bounds (Tensor): A tensor containing the lower and upper bounds.

    Returns:
        Tensor: The clamped tensor.
    """
    if bounds.dim() == 2 and bounds.size(0) == 2:
        # Common bounds for all samples: bounds shape is (2, dim)
        lower_bounds, upper_bounds = bounds
    elif bounds.dim() == 3 and bounds.size(1) == 2:
        # Per-sample bounds: bounds shape is (batch_size, 2, dim)
        lower_bounds = bounds[:, 0, :]
        upper_bounds = bounds[:, 1, :]
    else:
        raise ValueError("Invalid bounds dimension. Expected bounds of shape (2, dim) or (batch_size, 2, dim).")
    
    # Perform clamping
    return torch.max(torch.min(tensor, upper_bounds), lower_bounds)



In [3]:


def generate_initial_candidates(bounds: torch.Tensor, num_candidates: int) -> torch.Tensor:
    """
    Generates initial candidate points uniformly within the given bounds.

    Args:
        bounds (Tensor): A tensor containing the lower and upper bounds.
            - If bounds has shape (2, dim), it represents common bounds for all samples.
            - If bounds has shape (batch_size, 2, dim), it represents per-sample bounds.
        num_candidates (int): The number of candidates to generate.
            - For per-sample bounds, num_candidates must equal batch_size.

    Returns:
        Tensor: A tensor of initial candidate points with shape (num_candidates, dim).
    """
    
    
    if bounds.dim() == 2 and bounds.size(0) == 2:
        # Common bounds for all samples
        dim = bounds.shape[1]
        lower_bounds, upper_bounds = bounds  # Shapes: (dim,)
        # Generate random points uniformly within bounds
        random_numbers = torch.rand(num_candidates, dim)
        initial_points = lower_bounds + (upper_bounds - lower_bounds) * random_numbers
    elif bounds.dim() == 3 and bounds.size(1) == 2:
        # Per-sample bounds
        batch_size = bounds.shape[0]
        dim = bounds.shape[2]
        if num_candidates != batch_size:
            raise ValueError("For per-sample bounds, num_candidates must equal batch_size.")
        lower_bounds = bounds[:, 0, :]  # Shape: (batch_size, dim)
        upper_bounds = bounds[:, 1, :]  # Shape: (batch_size, dim)
        # Generate random points uniformly within per-sample bounds
        random_numbers = torch.rand(num_candidates, dim)
        initial_points = lower_bounds + (upper_bounds - lower_bounds) * random_numbers
    else:
        raise ValueError("Invalid bounds shape. Expected shape (2, dim) or (batch_size, 2, dim).")
    return initial_points


In [6]:
def optimize_acquisition_function(
    acquisition_function: AcquisitionFunction,
    bounds: Tensor,
    num_restarts: int,
    raw_samples: int,
    num_candidates: int,
    optimizer_options: Optional[Dict[str, Any]] = None,
) -> Tensor:
    """
    Optimizes the acquisition function to find the next set of candidates.

    Args:
        acquisition_function (AcquisitionFunction): The acquisition function to optimize.
        bounds (Tensor): A tensor containing the lower and upper bounds.
        num_restarts (int): Number of restarts for optimization.
        raw_samples (int): Number of raw samples to use for initialization.
        num_candidates (int): Number of candidates to generate.
        optimizer_options (Optional[Dict[str, Any]]): Options for the optimizer.

    Returns:
        Tensor: The optimized candidate points.
    """
    from botorch.optim.optimize import optimize_acqf

    # Use BoTorch's built-in optimizer for acquisition functions
    # https://botorch.org/api/optim.html
    candidates, _ = optimize_acqf(
        acq_function=acquisition_function,
        bounds=bounds,
        q=num_candidates,
        num_restarts=num_restarts,
        raw_samples=raw_samples,
        options=optimizer_options or {},
    )
    return candidates
